# Random Search for Hyperparameter Optimization 

There are lots of different search strategies that you can use for hyperparmeter-optimization. Bergstra and Bengio showed that random search is an effective method. 

As with any hyperparameter search method, you will compare models based on validation set performance, then choose a final architecture from that, and finally measure your test set performance. 


## K-fold Validation & Hyperparameter Search

This may be stating the obvious, but I found it useful for myself to understand how to integrate the hyperparameter search with k-fold validation. The pseudocode below is how I do it:

In [ ]:
'''

Search_loop= 1 to num_searches(

	hyperparameters= random_selection 
	validation_performance= 0
	
	loop_fold= 1 to K(
	validation_performance += Model(fold, hyperparameters)
	)
	
	model_average= validation_performance/K
	save-to-csv(model_average, hyperparameters)

Select hyperparameters that gave best model average

'''

When deciding on your number of searches (num_searches), here are a few things to consider:

1. How much time you have
2. How many other models you have
3. How much things are improving over the runs

You can leave things running over night and then decide whether to continue, the next morning.

Here is the code I use for my hyperparameter search. Essentially, I'm embedding num_searches number of dictionaries into one big dictionary. Each dictionary contains a unique set of hyperparameters. 

I'm using numpy to sample numbers randomly from logspace, there are others ways to randomly sample number, but this method is especially simple. 


In [ ]:
def hyperparam_search(num_searches):
    hyperparam_dict={}

    for j in range(num_searches):
        ## Randomely select hyperparameters
        dict_name= 'parameter_set_%i' %j
        hyperparam_dict[dict_name]={}

        #Hidden Units
        num_hidden_units= np.random.choice(np.logspace(start=2.1, stop=3, num=num_searches))
        #chooses between 128 to 1000 hidden units, uniformly distributed along log space
        rounded_hidden_units= np.round(num_hidden_units)
        hyperparam_dict[dict_name]['hidden_units']= rounded_hidden_units

        #Number of Layers
        num_layers= np.random.uniform(low=1, high=3)
        layer= np.round(num_layers)
        hyperparam_dict[dict_name]['num_layer']=layer

        #Learning Rate
        learning_rate= np.random.choice(np.logspace(start=-5, stop=0, num=num_searches))
        #Choose between 0.00001 and 1
        rounded_learning_rate = np.around(learning_rate, decimals=3)
        hyperparam_dict[dict_name]['learning_rate']= rounded_learning_rate
        
        #Batch Size
        batch_size= np.random.randint(low=64, high=200)
        hyperparam_dict[dict_name]['batch_size']= batch_size

    return hyperparam_dict

dict= hyperparam_search(num_searches)
print dict

When you print out the embedded dictionary, it gives you all the sets of parameters. You can store this dictionary and then call it when you're doing K-fold validation for your model. 

My print out looked like this, note that it sorted the dictionary randomly: 

{'parameter_set_2': {'learning_rate': 1.0000000000000001e-05, 'num_layer': 1.0, 'hidden_units': 355.0, 'batch_size': 169}, 
<br>'parameter_set_3': {'learning_rate': 1.0, 'num_layer': 2.0, 'hidden_units': 126.0, 'batch_size': 185}, 
<br>'parameter_set_0': {'learning_rate': 1.0, 'num_layer': 2.0, 'hidden_units': 1000.0, 'batch_size': 131}, 
<br>'parameter_set_1': {'learning_rate': 00.00316, 'num_layer': 1.0, 'hidden_units': 1000.0, 'batch_size': 135}, 
<br>'parameter_set_4': {'learning_rate': 0.00017782794100389227, 'num_layer': 1.0, 'hidden_units': 596.0, 'batch_size': 69}}


